In [ ]:
import numpy as np 
from scipy.linalg import expm 
import matplotlib.pyplot as plt 
from random import choices, seed 
from joblib import Parallel, delayed
from main.tomography import Hamiltonian_Learning, Pauli_Transfer_Matrix
from main.randomized_compiling import RandomizedCompiling
import json
from qiskit_ibm_runtime import RuntimeEncoder, RuntimeDecoder

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
service = QiskitRuntimeService(channel='ibm_cloud', 
                                token='')
backend = service.backend('ibm_brisbane')
coupling = backend.configuration().coupling_map

qubits_layaout = [13,12,11,10,9,8,7,6,5,4,3,2,1,0,14,
                    18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,36,
                    51,50,49,48,47,46,45,44,43,42,41,40,39,38,37,52,
                    56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,74,
                    89,88,87,86,85,84,83,82,81,80,79,78,77,76,75,90,
                    94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,112,
                    126,125,124,123,122,121,120,119,118,117,116,115,114,113] 

We load the Qiskit results. 

In [ ]:
with open("datos_100q_RC.json", "r") as file:
    results = json.load(file, cls=RuntimeDecoder) 

results

Setting the functions for the postprocessing of the Hamiltonian Learning protocol.

In [ ]:
num_qubits = 109
hs = [ 1/8 for j in range(num_qubits) ] # parameters h_j
Js = [ 1/16 for j in range(num_qubits-1) ] # Parameters J_j
hamil_learn = Hamiltonian_Learning( num_qubits )
qubits_tomo = hamil_learn.groups_qubits_tomo()

In [ ]:
I = np.eye(2)
Z = np.diag([1,-1])
X = np.array([[0,1],[1,0]])
H = (1/8)*np.kron(I,Z) + (1/8)*np.kron(Z,I) + (1/16)*np.kron(X,X)
U = expm( -1j*H )

In [ ]:
probs = [ prob.data.c.get_counts() for prob in results ]
n_shots = int( np.sum([ list(prob.values()) for prob in probs ]) )
n_shots

In [ ]:
qubit_coords = {
    # (index: (x, y)) coordinates matching the layout you uploaded
    0: (0, 0), 1: (1, 0), 2: (2, 0), 3: (3, 0), 4: (4, 0), 5: (5, 0), 6: (6, 0),
    7: (7, 0), 8: (8, 0), 9: (9, 0), 10: (10, 0), 11: (11, 0), 12: (12, 0), 13: (13, 0),
    14: (0, 1), 15: (4, 1), 16: (8, 1), 17: (12, 1),
    18: (0, 2), 19: (1, 2), 20: (2, 2), 21: (3, 2), 22: (4, 2), 23: (5, 2), 24: (6, 2),
    25: (7, 2), 26: (8, 2), 27: (9, 2), 28: (10, 2), 29: (11, 2), 30: (12, 2), 31: (13, 2),
    32: (14, 2), 33: (2, 3), 34: (6, 3), 35: (10, 3), 36: (14, 3), 37: (0, 4), 38: (1, 4),
    39: (2, 4), 40: (3, 4), 41: (4, 4), 42: (5, 4), 43: (6, 4), 44: (7, 4), 45: (8, 4),
    46: (9, 4), 47: (10, 4), 48: (11, 4), 49: (12, 4), 50: (13, 4), 51: (14, 4), 52: (0, 5),
    53: (4, 5), 54: (8, 5), 55: (12, 5), 56: (0, 6), 57: (1, 6), 58: (2, 6), 59: (3, 6),
    60: (4, 6), 61: (5, 6), 62: (6, 6), 63: (7, 6), 64: (8, 6), 65: (9, 6), 66: (10, 6),
    67: (11, 6), 68: (12, 6), 69: (13, 6), 70: (14, 6), 71: (2, 7), 72: (6, 7), 73: (10, 7),
    74: (14, 7), 75: (0, 8), 76: (1, 8), 77: (2, 8), 78: (3, 8), 79: (4, 8), 80: (5, 8),
    81: (6, 8), 82: (7, 8), 83: (8, 8), 84: (9, 8), 85: (10, 8), 86: (11, 8), 87: (12, 8),
    88: (13, 8), 89: (14, 8), 90: (0, 9), 91: (4, 9), 92: (8, 9), 93: (12, 9), 94: (0, 10),
    95: (1, 10), 96: (2, 10), 97: (3, 10), 98: (4, 10), 99: (5, 10), 100: (6, 10), 101: (7, 10),
    102: (8, 10), 103: (9, 10), 104: (10, 10), 105: (11, 10), 106: (12, 10), 107: (13, 10),
    108: (14, 10), 109: (2, 11), 110: (6, 11), 111: (10, 11), 112: (14, 11), 113: (1, 12),
    114: (2, 12), 115: (3, 12), 116: (4, 12), 117: (5, 12), 118: (6, 12), 119: (7, 12),
    120: (8, 12), 121: (9, 12), 122: (10, 12), 123: (11, 12), 124: (12, 12), 125: (13, 12),
    126: (14, 12),
}

We execute bootstrapping method to estimate the error of the experimentally estimated parameters.

In [ ]:
def bootstrap_resample( probs , j =1 ):
    
    seed(j)
    sampled_probs = []
    for prob in probs:
        probs_w = list(prob.values())
        probs_w =np.array(probs_w) / np.sum(probs_w)
        sampled_keys = choices( list(prob.keys()), weights=probs_w,
                                k=int( np.sum(list(prob.values())) ) )
        sampled_prob = {  key:prob[key] for key in sampled_keys }
        sampled_probs.append( sampled_prob )

    hamil_learn = Hamiltonian_Learning( num_qubits )
    qubits_tomo = hamil_learn.groups_qubits_tomo()
    Hs = hamil_learn.hamiltonian_from_tomo( sampled_probs, init=U ) 
    lambdaa = hamil_learn.noise()

    qubits_tomo = np.array( qubits_tomo ).reshape(-1,2)
    params_local = { str(j):[] for j in qubits_layaout }
    errors_local = { str(j):[] for j in qubits_layaout }
    params_two   = {}
    errors_two   = {}
    for j, (q1, q2) in enumerate(qubits_tomo):
        pauli_coef_tomo = Pauli_Transfer_Matrix( Hs[j], num_qubits=2 )  
        params_local[str(qubits_layaout[q1])].append( pauli_coef_tomo[0,-1] )
        params_local[str(qubits_layaout[q2])].append( pauli_coef_tomo[-1,0] )
        params_two[str((qubits_layaout[q1],qubits_layaout[q2]))]= pauli_coef_tomo[1,1]
        params_two[str((qubits_layaout[q2],qubits_layaout[q1]))]= pauli_coef_tomo[1,1]

        errors_local[str(qubits_layaout[q1])].append( abs(1/8-pauli_coef_tomo[0,-1]) )
        errors_local[str(qubits_layaout[q2])].append( abs(1/8-pauli_coef_tomo[-1,0]) )
        errors_two[str((qubits_layaout[q1],qubits_layaout[q2]))]= abs(1/16-pauli_coef_tomo[1,1])
        errors_two[str((qubits_layaout[q2],qubits_layaout[q1]))]= abs(1/16-pauli_coef_tomo[1,1])

    params_local = { j : np.mean(params_local[j]) for j in params_local }
    errors_local = { j : np.mean(errors_local[j]) for j in errors_local }

    params = { 'params':{'h': params_local,
                        'J': params_two },
        'error' : {'h' : errors_local,
                        'J' : errors_two}, 
        'coupling' : coupling,
        'shots'  :  n_shots, 
        'lambda' : lambdaa, 
                }
    
    return params 

In [ ]:
params_mc = Parallel(n_jobs=10)( delayed(bootstrap_resample)( probs, j ) 
                                for j in range(10000)  )
#1000-->70m

We create a new Json file with the postprocessed results.

In [ ]:
h_mc = []
J_mc = []
eh_mc = []
eJ_mc = []
L_mc = []

for params in params_mc:
    h_mc.append( params['params']['h'] )
    J_mc.append( params['params']['J'] )
    eh_mc.append( params['error']['h'] )
    eJ_mc.append( params['error']['J'] )
    L_mc.append( params['lambda'] )

In [ ]:
def mean_dict( dicts ):
    dict_new = {}
    for key in dicts[0].keys():
        dict_new[key] = ( float(np.mean([d[key] for d in dicts ])),
                            float(np.std([d[key] for d in dicts ])) )
    return dict_new 

In [ ]:
params_local  = mean_dict( h_mc )
params_two  = mean_dict( J_mc )
errors_local = mean_dict( eh_mc )
errors_two = mean_dict( eJ_mc )
lambdaa = np.mean( L_mc )
lambdaa_std = np.std( L_mc )

params = { 'params':{'h': params_local,
                        'J': params_two},
        'error' : {'h' : errors_local,
                        'J' : errors_two}, 
        'coupling' : coupling,
        'shots'  :  n_shots, 
        'lambda' : (lambdaa,lambdaa_std), 
                }

In [ ]:
with open("params_100q_RC_mc10000.json", "w") as file:
        json.dump(params, file )